In [48]:
#Importamos 4 bibliotecas:
#BeautifulSoup para lidar com html
from bs4 import BeautifulSoup
#Requests para obter dados de uma pagina na web
import requests
#google.generativeai que é a API da IA generativa da Google


In [34]:
#Configuração da IA:

#inserção da chave individual para a API
genai.configure(api_key='Sua chave')
#Configurando qual modelo do gemini vamos usar, o gemini-pro é a mais adequada para textos:
model = genai.GenerativeModel('gemini-pro')
#Definindo a "temperatura" da IA, podemos inserir números de 0.0 a 1.0, quanto mais próximo de 1 mais criativa a IA será
genai.GenerationConfig(temperature = 0.0)

GenerationConfig(candidate_count=None, stop_sequences=None, max_output_tokens=None, temperature=0.0, top_p=None, top_k=None)

In [35]:
#Esse é um link do site vagas.com, de uma pesqisa com vagas para analista de dados, aqui temos a primeira página da pesquisa

link='https://www.vagas.com.br/vagas-de-analista-de-dados'

In [36]:
#Com a o método get da biblioteca request, temos o html da página
conteudo=requests.get(link)

In [37]:
soup=BeautifulSoup(conteudo.text,'html.parser')
#Na página que temos do vagas, ainda não temos a descrição, a página de pesquisa é um página com links para a descrição das vagas, precisamos desse link
links_vagas=[]
#dentro do atributo "a" temos a classe link-detalhes-vaga, é lá que estão o link para a descrição das vagas
for link in soup.find_all("a", {"class":"link-detalhes-vaga"}):
    links_vagas.append(link.get('href'))
#Adicionamos os links a uma lista e imprimimos:    
print(links_vagas)

['/vagas/v2617418/analista-de-dados', '/vagas/v2621511/analista-dados-sr', '/vagas/v2621516/analista-dados-sr', '/vagas/v2621983/analista-de-dados', '/vagas/v2619152/analista-de-dados-pleno', '/vagas/v2621787/analista-de-dados-jr', '/vagas/v2623827/analista-de-dados-senior', '/vagas/v2617428/analista-jr-de-banco-de-dados', '/vagas/v2618239/analista-de-dados-devops', '/vagas/v2618171/analista-de-sistemas-dados', '/vagas/v2622362/analista-de-dados-clinicos-ii', '/vagas/v2624427/analista-de-processo-e-dados-junior', '/vagas/v2625002/analista-de-infraestrutura-de-dados', '/vagas/v2617984/analista-de-dados-em-power-bi', '/vagas/v2623328/analista-de-banco-de-dados-dba', '/vagas/v2617417/dba-oracle-analista-de-banco-de-dados', '/vagas/v2627425/analista-de-laboratorio-senior-temporario-integridade-de-dados', '/vagas/v2624875/analista-dados-mestre-pleno-afirmativa-para-pcd', '/vagas/v2624154/analista-de-recursos-humanos-jr-dados-indicadores-e-governanca', '/vagas/v2627111/analista-de-dados-seni

In [38]:
#criamos uma string para armazenar os resultados
texto_resultado=" "
for link in links_vagas:
    #os links da lista no item anterior estão incompletos, vamos completar e acessar o html:
    link3="https://www.vagas.com.br"+link
    conteudo=requests.get(link3)
    soup2=BeautifulSoup(conteudo.text,'html.parser')
    #Dentro do html vamos pegar a area onde tem a descrição da vaga
    texto=(soup2.find_all("div", {"data-testid":"JobDescription"}))
    #Para processar melhor a resposta, foi substituida a tag span que apaerecia no resultado
    y=str(texto).replace("</span>"," ")
    z=BeautifulSoup(y,'html.parser')
    #Pedi para a IA retirar as hard skills de dentro da descrição da vaga, e colocar ela de forma resumida, além disso o resultado precisa ser semelhante a uma lista em python
    response = model.generate_content("com base no texto a seguir, e lembrando que é proibido colocar hifen ou traço na resposta, separe as hard skills que o profissional deve ter, resumindo a hard skill com até 4 palavras e deixando apenas uma hard Skill por linha, se tiver mais de uma ferramenta na mesma linha, separe e coloque em outra linha. Ex: R/python, deve ser separado e estar R em uma linha e python em outra. Não coloque palavras extras, a resposta deve ser uma lista em python, com os itens separados por virgula e cada linha entre aspas, não esqueça dos colchetes"+z.get_text())
    #Adicionamos a variavel de resultado
    texto_resultado=texto_resultado+","+response.text

In [39]:
#Como resultado temos:
print(texto_resultado)

 ,["Análise de Dados",
 "Desenvolvimento de Queries",
 "Mineração de Dados",
 "Previsão de Datas Comemorativas",
 "Automação de Processos",
 "Desenvolvimento de Relatórios",
 "Dashboard e Gráficos",
 "Integração de Dados",
 "Machine Learning",
 "Otimização de Processos ETL",
 "Manutenção de Views",
 "Refatoração de Códigos",
 "Extração de Dados via API",
 "Documentação de Linhagens",
 "Criação de Dicionário de Dados",
 "Análise de Dados com Power BI",
 "Colaboração Estratégica"],[
"PowerBI",
"SQL",
"Modelagem de Dados OLTP",
"Modelagem de Dados OLAP",
"Conceitos Datalake",
"Conceitos Lakehouse",
"ETL/ELT",
"Metodologia Ágil",
"Inglês leitura"
],[
"PowerBI", 
"SQL", 
"Modelagem de Dados OLTP", 
"Modelagem de Dados OLAP", 
"Conceitos Datalake", 
"Conceitos Lakehouse", 
"ETL/ELT", 
"Metodologia Ágil", 
"Inglês leitura"
],["Power BI",
 "Excel avançado"],[
"Power BI",
"Excel",
"SQL",
"Python",
"VBA",
"TOTVS"
],["ETL / ELT", "Construção de dataset", "Governança de objetos", "Manutenção de ba

In [40]:
#Temos algo semelhante a uma lista de lista, porém, tem um virgula a mais no inicio, vamos retirar ela:
Lista_texto=texto_resultado[2:]

In [41]:
# A IA também pulou linha, o que não é interessante para a análise, por isso vamos retirar as quebras de linha:
Lista_texto =(Lista_texto).replace('\n', '')
#Agora podemos efetivamente transformar em uma lista de listas:
Lista_texto=list(eval(Lista_texto))

#Como resultado:
print(Lista_texto)

[['Análise de Dados', 'Desenvolvimento de Queries', 'Mineração de Dados', 'Previsão de Datas Comemorativas', 'Automação de Processos', 'Desenvolvimento de Relatórios', 'Dashboard e Gráficos', 'Integração de Dados', 'Machine Learning', 'Otimização de Processos ETL', 'Manutenção de Views', 'Refatoração de Códigos', 'Extração de Dados via API', 'Documentação de Linhagens', 'Criação de Dicionário de Dados', 'Análise de Dados com Power BI', 'Colaboração Estratégica'], ['PowerBI', 'SQL', 'Modelagem de Dados OLTP', 'Modelagem de Dados OLAP', 'Conceitos Datalake', 'Conceitos Lakehouse', 'ETL/ELT', 'Metodologia Ágil', 'Inglês leitura'], ['PowerBI', 'SQL', 'Modelagem de Dados OLTP', 'Modelagem de Dados OLAP', 'Conceitos Datalake', 'Conceitos Lakehouse', 'ETL/ELT', 'Metodologia Ágil', 'Inglês leitura'], ['Power BI', 'Excel avançado'], ['Power BI', 'Excel', 'SQL', 'Python', 'VBA', 'TOTVS'], ['ETL / ELT', 'Construção de dataset', 'Governança de objetos', 'Manutenção de banco de dados', 'Inovação e 

In [42]:
#A IA retornou algo parecido como uma lista de listas, mas será mais interessante uma lista unica:
lista_final = sum(Lista_texto, [])
print(lista_final)

['Análise de Dados', 'Desenvolvimento de Queries', 'Mineração de Dados', 'Previsão de Datas Comemorativas', 'Automação de Processos', 'Desenvolvimento de Relatórios', 'Dashboard e Gráficos', 'Integração de Dados', 'Machine Learning', 'Otimização de Processos ETL', 'Manutenção de Views', 'Refatoração de Códigos', 'Extração de Dados via API', 'Documentação de Linhagens', 'Criação de Dicionário de Dados', 'Análise de Dados com Power BI', 'Colaboração Estratégica', 'PowerBI', 'SQL', 'Modelagem de Dados OLTP', 'Modelagem de Dados OLAP', 'Conceitos Datalake', 'Conceitos Lakehouse', 'ETL/ELT', 'Metodologia Ágil', 'Inglês leitura', 'PowerBI', 'SQL', 'Modelagem de Dados OLTP', 'Modelagem de Dados OLAP', 'Conceitos Datalake', 'Conceitos Lakehouse', 'ETL/ELT', 'Metodologia Ágil', 'Inglês leitura', 'Power BI', 'Excel avançado', 'Power BI', 'Excel', 'SQL', 'Python', 'VBA', 'TOTVS', 'ETL / ELT', 'Construção de dataset', 'Governança de objetos', 'Manutenção de banco de dados', 'Inovação e cultura de 

In [43]:
#Agora criamos um dicionário para agrupar as palavras iguais e assim vermos os itens mais requisitados
dicionario=dict()
for item in lista_final:
        if item not in dicionario:
            dicionario[item] = 1
        else:
            dicionario[item] += 1
            
print(dicionario)

{'Análise de Dados': 2, 'Desenvolvimento de Queries': 1, 'Mineração de Dados': 1, 'Previsão de Datas Comemorativas': 1, 'Automação de Processos': 1, 'Desenvolvimento de Relatórios': 1, 'Dashboard e Gráficos': 1, 'Integração de Dados': 1, 'Machine Learning': 1, 'Otimização de Processos ETL': 1, 'Manutenção de Views': 1, 'Refatoração de Códigos': 1, 'Extração de Dados via API': 1, 'Documentação de Linhagens': 1, 'Criação de Dicionário de Dados': 1, 'Análise de Dados com Power BI': 1, 'Colaboração Estratégica': 1, 'PowerBI': 2, 'SQL': 8, 'Modelagem de Dados OLTP': 2, 'Modelagem de Dados OLAP': 2, 'Conceitos Datalake': 2, 'Conceitos Lakehouse': 2, 'ETL/ELT': 2, 'Metodologia Ágil': 2, 'Inglês leitura': 2, 'Power BI': 9, 'Excel avançado': 3, 'Excel': 10, 'Python': 4, 'VBA': 1, 'TOTVS': 2, 'ETL / ELT': 1, 'Construção de dataset': 1, 'Governança de objetos': 1, 'Manutenção de banco de dados': 1, 'Inovação e cultura de dados': 1, 'Estabelecimento e defesa de melhores práticas': 1, 'RD CRM': 1, 

In [44]:
#Ordenando o dicionário

dicionario_ordenado = dict(sorted(dicionario.items(), key=lambda item: item[1], reverse=True))
print(dicionario_ordenado)

{'Excel': 10, 'Power BI': 9, 'SQL': 8, 'Power Point': 5, 'Python': 4, 'Excel avançado': 3, 'SQL Server': 3, 'Pacote Office': 3, 'Excel Avançado': 3, 'Análise de Dados': 2, 'PowerBI': 2, 'Modelagem de Dados OLTP': 2, 'Modelagem de Dados OLAP': 2, 'Conceitos Datalake': 2, 'Conceitos Lakehouse': 2, 'ETL/ELT': 2, 'Metodologia Ágil': 2, 'Inglês leitura': 2, 'TOTVS': 2, 'Oracle': 2, 'Análise de dados': 2, 'ETL': 2, 'R': 2, 'SAP': 2, 'Desenvolvimento de Queries': 1, 'Mineração de Dados': 1, 'Previsão de Datas Comemorativas': 1, 'Automação de Processos': 1, 'Desenvolvimento de Relatórios': 1, 'Dashboard e Gráficos': 1, 'Integração de Dados': 1, 'Machine Learning': 1, 'Otimização de Processos ETL': 1, 'Manutenção de Views': 1, 'Refatoração de Códigos': 1, 'Extração de Dados via API': 1, 'Documentação de Linhagens': 1, 'Criação de Dicionário de Dados': 1, 'Análise de Dados com Power BI': 1, 'Colaboração Estratégica': 1, 'VBA': 1, 'ETL / ELT': 1, 'Construção de dataset': 1, 'Governança de objetos